In [9]:
import pandas as pd
import numpy as np
from keras.layers import Activation, Dense, Dropout, SpatialDropout1D,Input,Masking,Bidirectional, TimeDistributed
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM,GRU
from keras.models import Sequential, Model
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from random import seed
from sklearn.metrics import roc_auc_score
#seed(1)
import tensorflow as tf
from keras.layers import Input, Dense
from keras.models import Model
import scipy.stats
from prettytable import PrettyTable
import math
import itertools
from random import shuffle


#defining the early stopping criteria
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,restore_best_weights=True,patience=3)

gs = pd.read_csv('lstm_analysis.csv')
gs['urine_per_kg_hour'] = gs['urine_per_hour']/gs['currentdateweight']
gs.drop('Unnamed: 0',axis=1,inplace=True)
gs.drop('Unnamed: 0.1',axis=1,inplace=True)



def split_70(x):
    return int((round((x/15)*0.7))*15)


def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return round(m,3), round(m-h,3), round(m+h,3)



def range_finder(x):
    length = x
    fractional = (x/15.0) - math.floor(x/15.0)
    return int(round(fractional*15))

def make_lstm(gd):


    final_df = pd.DataFrame(columns=gd.columns)
    ids = gd.uhid.unique()
    shuffle(ids)
    for i in ids:
        x = gd[gd['uhid']==i]
        x = x[range_finder(len(x)):len(x)]

        final_df = final_df.append(x,ignore_index=True)



    final_df.fillna(-999,inplace=True)


    # In[ ]:


    death_cases = final_df[final_df.dischargestatus == 1]
    discharge_cases = final_df[final_df.dischargestatus == 0]
    
    train_death = death_cases[:split_70(len(death_cases))]
    test_death = death_cases[split_70(len(death_cases)):]
    
    train_discharge = discharge_cases[:split_70(len(discharge_cases))]
    test_discharge = discharge_cases[split_70(len(discharge_cases)):]
    
    train = pd.concat([train_death,train_discharge])
    test = pd.concat([test_death,test_discharge])



    # In[ ]:


    y_train = train['dischargestatus']
    X_train = train.drop('dischargestatus',axis=1)
    X_train = X_train.drop('uhid',axis=1)
    #X_train = X_train.drop('visittime',axis=1)

    y_test = test['dischargestatus']
    X_test = test.drop('dischargestatus',axis=1)
    X_test = X_test.drop('uhid',axis=1)
    #X_test = X_test.drop('startdate',axis=1)


    # In[ ]:

    auc_roc_inter = []
    val_a = []
    train_a = []


    #converting the data into a numpy array
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    ytrain1 = []
    for i in range(0,len(y_train),15):
        #print(i)
        y1 = y_train[i:i+15]
        ytrain1.append(y1[-1])

    ytest1 = []
    for i in range(0,len(y_test),15):
        #print(i)
        y1 = y_test[i:i+15]
        ytest1.append(y1[-1])

    ytrain1 = np.array(ytrain1)
    ytest1 = np.array(ytest1)

    Xtrain = np.reshape(X_train, (-1, 15, X_train.shape[1]))
    Xtest = np.reshape(X_test, (-1, 15, X_test.shape[1]))

    return Xtrain,Xtest,ytrain1,ytest1

def lstm_model(n,Xtrain,Xtest,ytrain1,ytest1):
    auc_roc_inter = []
    val_a = []
    train_a = []
    for i in range(2):
        #Building the LSTM model
        X = Input(shape=(None, n), name='X')
        mX = Masking()(X)
        lstm = Bidirectional(LSTM(units=512,activation='tanh',return_sequences=True,recurrent_dropout=0.5,dropout=0.3))
        mX = lstm(mX)
        L = LSTM(units=64,activation='tanh',return_sequences=False)(mX)
        y = Dense(1, activation="sigmoid")(L)
        outputs = [y]
        inputs = [X]
        model = Model(inputs,outputs)
        model.compile(loss="binary_crossentropy",optimizer='adam',metrics=['accuracy'])

        v_a = []
        t_a = []
        #fitting the model
        model.fit(Xtrain, ytrain1, batch_size=60 ,validation_split=0.15,epochs=38,callbacks=[es])
        #history = model.fit(Xtrain, ytrain1, batch_size=60 ,validation_split=0.15,epochs=38,callbacks=[es])
        for i in range(len(model.history.history['val_accuracy'])):
            v_a.append(model.history.history['val_accuracy'][i])
            t_a.append(model.history.history['accuracy'][i])
        #predictions
        y_pred = model.predict(Xtest)
        #y_pred = y_pred.round()
        y_test = np.array(ytest1)
        y_pred = np.array(y_pred)
        y_test = pd.DataFrame(y_test)
        y_test = np.array(y_test)

        def acc(x):
            if x>0.5:
                return 1
            else:
                return 0

        y_model=[]
        for i in y_pred:
            y_model.append(acc(i))
        y_answer=[]
        for j in y_test:
            y_answer.append(acc(j))
            
        val_a.append(v_a)
        train_a.append(t_a)
        auc_roc_inter.append(roc_auc_score(y_answer,y_pred))
        continue
    
    
        
    return auc_roc_inter,y_model,y_answer




/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (23,39,47,48,49,50,54,55,57,70,71,72,73,83,88,92,93,94,102,103) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (15,21,23,31,37,39,47,48,49,50,54,55,57,59,65,66,70,71,72,73,75,83,88,92,93,94,102,103) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
cols_to_use = ['uhid','currentdateweight','dischargestatus']
gd = gs[cols_to_use]
Xtrain,Xtest,ytrain1,ytest1 = make_lstm(gd)
an,y_model,y_answer = lstm_model((len(cols_to_use)-2),Xtrain,Xtest,ytrain1,ytest1)
a_a = mean_confidence_interval(an)

Train on 38816 samples, validate on 6850 samples
Epoch 1/38
38816/38816 [==============================] - 526s 14ms/step - loss: 0.6176 - accuracy: 0.6628 - val_loss: 0.9215 - val_accuracy: 0.1985
Epoch 2/38
38816/38816 [==============================] - 453s 12ms/step - loss: 0.5943 - accuracy: 0.6977 - val_loss: 0.9718 - val_accuracy: 0.1984
Epoch 3/38
38816/38816 [==============================] - 441s 11ms/step - loss: 0.5905 - accuracy: 0.7046 - val_loss: 0.9224 - val_accuracy: 0.1984
Epoch 4/38
38816/38816 [==============================] - 476s 12ms/step - loss: 0.5879 - accuracy: 0.7061 - val_loss: 1.0044 - val_accuracy: 0.1562
Restoring model weights from the end of the best epoch
Epoch 00004: early stopping
Train on 38816 samples, validate on 6850 samples
Epoch 1/38
38816/38816 [==============================] - 446s 11ms/step - loss: 0.6220 - accuracy: 0.6507 - val_loss: 0.8722 - val_accuracy: 0.3248
Epoch 2/38
38816/38816 [==============================] - 382s 10ms/step -

In [ ]:
a_a

In [9]:
cols_to_use = ['uhid','currentdateheight','dischargestatus']
gd = gs[cols_to_use]
Xtrain,Xtest,ytrain1,ytest1 = make_lstm(gd)
an,y_model,y_answer = lstm_model((len(cols_to_use)-2),Xtrain,Xtest,ytrain1,ytest1)
a_b = mean_confidence_interval(an)

Train on 2263 samples, validate on 400 samples
Epoch 1/38
2263/2263 [==============================] - 9s 4ms/step - loss: 0.4239 - accuracy: 0.8414 - val_loss: 0.4355 - val_accuracy: 0.7950
Epoch 2/38
2263/2263 [==============================] - 6s 3ms/step - loss: 0.3915 - accuracy: 0.8590 - val_loss: 0.5323 - val_accuracy: 0.7950
Epoch 3/38
2263/2263 [==============================] - 6s 3ms/step - loss: 0.3749 - accuracy: 0.8590 - val_loss: 0.5607 - val_accuracy: 0.7950
Epoch 4/38
2263/2263 [==============================] - 6s 3ms/step - loss: 0.3756 - accuracy: 0.8590 - val_loss: 0.5149 - val_accuracy: 0.7950
Restoring model weights from the end of the best epoch
Epoch 00004: early stopping
Train on 2263 samples, validate on 400 samples
Epoch 1/38
2263/2263 [==============================] - 9s 4ms/step - loss: 0.4194 - accuracy: 0.8338 - val_loss: 0.5681 - val_accuracy: 0.7950
Epoch 2/38
2263/2263 [==============================] - 6s 3ms/step - loss: 0.3803 - accuracy: 0.8590 -

In [10]:
cols_to_use = ['uhid','ph','dischargestatus']
gd = gs[cols_to_use]
Xtrain,Xtest,ytrain1,ytest1 = make_lstm(gd)
an,y_model,y_answer = lstm_model((len(cols_to_use)-2),Xtrain,Xtest,ytrain1,ytest1)
a_c = mean_confidence_interval(an)

Train on 2263 samples, validate on 400 samples
Epoch 1/38
2263/2263 [==============================] - 9s 4ms/step - loss: 0.3940 - accuracy: 0.8308 - val_loss: 0.2259 - val_accuracy: 0.8350
Epoch 2/38
2263/2263 [==============================] - 6s 3ms/step - loss: 0.3681 - accuracy: 0.8515 - val_loss: 0.2600 - val_accuracy: 0.8350
Epoch 3/38
2263/2263 [==============================] - 6s 3ms/step - loss: 0.3595 - accuracy: 0.8573 - val_loss: 0.2575 - val_accuracy: 0.8375
Epoch 4/38
2263/2263 [==============================] - 6s 3ms/step - loss: 0.3670 - accuracy: 0.8506 - val_loss: 0.2341 - val_accuracy: 0.8375
Restoring model weights from the end of the best epoch
Epoch 00004: early stopping
Train on 2263 samples, validate on 400 samples
Epoch 1/38
2263/2263 [==============================] - 10s 4ms/step - loss: 0.3813 - accuracy: 0.8484 - val_loss: 0.2246 - val_accuracy: 0.9525
Epoch 2/38
2263/2263 [==============================] - 6s 3ms/step - loss: 0.3644 - accuracy: 0.8524 

In [11]:
cols_to_use = ['uhid','tpn-tfl','dischargestatus']
gd = gs[cols_to_use]
Xtrain,Xtest,ytrain1,ytest1 = make_lstm(gd)
an,y_model,y_answer = lstm_model((len(cols_to_use)-2),Xtrain,Xtest,ytrain1,ytest1)
a_d = mean_confidence_interval(an)

Train on 2263 samples, validate on 400 samples
Epoch 1/38
2263/2263 [==============================] - 10s 4ms/step - loss: 0.6547 - accuracy: 0.8674 - val_loss: 0.6403 - val_accuracy: 0.8900
Epoch 2/38
2263/2263 [==============================] - 6s 3ms/step - loss: 0.6363 - accuracy: 0.8710 - val_loss: 0.6269 - val_accuracy: 0.8900
Epoch 3/38
2263/2263 [==============================] - 6s 3ms/step - loss: 0.6186 - accuracy: 0.8657 - val_loss: 0.6017 - val_accuracy: 0.8900
Epoch 4/38
2263/2263 [==============================] - 6s 3ms/step - loss: 0.6114 - accuracy: 0.8692 - val_loss: 0.5954 - val_accuracy: 0.8900
Epoch 5/38
2263/2263 [==============================] - 6s 3ms/step - loss: 0.5950 - accuracy: 0.8626 - val_loss: 0.6013 - val_accuracy: 0.8900
Epoch 6/38
2263/2263 [==============================] - 6s 3ms/step - loss: 0.5861 - accuracy: 0.8692 - val_loss: 0.5696 - val_accuracy: 0.8900
Epoch 7/38
2263/2263 [==============================] - 6s 3ms/step - loss: 0.5750 - acc

Epoch 19/38
2263/2263 [==============================] - 6s 3ms/step - loss: 0.4824 - accuracy: 0.8767 - val_loss: 0.4728 - val_accuracy: 0.8900
Epoch 20/38
2263/2263 [==============================] - 6s 3ms/step - loss: 0.4740 - accuracy: 0.8794 - val_loss: 0.4658 - val_accuracy: 0.8900
Epoch 21/38
2263/2263 [==============================] - 6s 3ms/step - loss: 0.4683 - accuracy: 0.8749 - val_loss: 0.4634 - val_accuracy: 0.8900
Epoch 22/38
2263/2263 [==============================] - 6s 3ms/step - loss: 0.4588 - accuracy: 0.8838 - val_loss: 0.4532 - val_accuracy: 0.8900
Epoch 23/38
2263/2263 [==============================] - 6s 3ms/step - loss: 0.4493 - accuracy: 0.8886 - val_loss: 0.4400 - val_accuracy: 0.8900
Epoch 24/38
2263/2263 [==============================] - 6s 3ms/step - loss: 0.4534 - accuracy: 0.8727 - val_loss: 0.4349 - val_accuracy: 0.8900
Epoch 25/38
2263/2263 [==============================] - 6s 3ms/step - loss: 0.4483 - accuracy: 0.8719 - val_loss: 0.4192 - val_ac

In [12]:
cols_to_use = ['uhid','rbs','dischargestatus']
gd = gs[cols_to_use]
Xtrain,Xtest,ytrain1,ytest1 = make_lstm(gd)
an,y_model,y_answer = lstm_model((len(cols_to_use)-2),Xtrain,Xtest,ytrain1,ytest1)
a_e = mean_confidence_interval(an)

Train on 2263 samples, validate on 400 samples
Epoch 1/38
2263/2263 [==============================] - 6s 2ms/step - loss: 0.4520 - accuracy: 0.8378 - val_loss: 0.2617 - val_accuracy: 0.9575
Epoch 2/38
2263/2263 [==============================] - 4s 2ms/step - loss: 0.4250 - accuracy: 0.8436 - val_loss: 0.3237 - val_accuracy: 0.9575
Epoch 3/38
2263/2263 [==============================] - 4s 2ms/step - loss: 0.4250 - accuracy: 0.8422 - val_loss: 0.2651 - val_accuracy: 0.9600
Epoch 4/38
2263/2263 [==============================] - 4s 2ms/step - loss: 0.4242 - accuracy: 0.8462 - val_loss: 0.3180 - val_accuracy: 0.9600
Restoring model weights from the end of the best epoch
Epoch 00004: early stopping
Train on 2263 samples, validate on 400 samples
Epoch 1/38
2263/2263 [==============================] - 6s 2ms/step - loss: 0.4457 - accuracy: 0.8427 - val_loss: 0.2087 - val_accuracy: 0.9600
Epoch 2/38
2263/2263 [==============================] - 4s 2ms/step - loss: 0.4253 - accuracy: 0.8462 -

In [13]:
cols_to_use = ['uhid','stool_day_total','dischargestatus']
gd = gs[cols_to_use]
Xtrain,Xtest,ytrain1,ytest1 = make_lstm(gd)
an,y_model,y_answer = lstm_model((len(cols_to_use)-2),Xtrain,Xtest,ytrain1,ytest1)
a_f = mean_confidence_interval(an)

Train on 2263 samples, validate on 400 samples
Epoch 1/38
2263/2263 [==============================] - 6s 2ms/step - loss: 0.2786 - accuracy: 0.8856 - val_loss: 0.7497 - val_accuracy: 0.6975
Epoch 2/38
2263/2263 [==============================] - 4s 2ms/step - loss: 0.2557 - accuracy: 0.9085 - val_loss: 0.7461 - val_accuracy: 0.7550
Epoch 3/38
2263/2263 [==============================] - 4s 2ms/step - loss: 0.2501 - accuracy: 0.9342 - val_loss: 0.6768 - val_accuracy: 0.7550
Epoch 4/38
2263/2263 [==============================] - 4s 2ms/step - loss: 0.2494 - accuracy: 0.9342 - val_loss: 0.6836 - val_accuracy: 0.7550
Epoch 5/38
2263/2263 [==============================] - 4s 2ms/step - loss: 0.2520 - accuracy: 0.9342 - val_loss: 0.7042 - val_accuracy: 0.7550
Epoch 6/38
2263/2263 [==============================] - 4s 2ms/step - loss: 0.2500 - accuracy: 0.9342 - val_loss: 0.7995 - val_accuracy: 0.7550
Restoring model weights from the end of the best epoch
Epoch 00006: early stopping
Train 

In [15]:
cols_to_use = ['uhid','urine_per_kg_hour','dischargestatus']
gd = gs[cols_to_use]
Xtrain,Xtest,ytrain1,ytest1 = make_lstm(gd)
an,y_model,y_answer = lstm_model((len(cols_to_use)-2),Xtrain,Xtest,ytrain1,ytest1)
a_g = mean_confidence_interval(an)

Train on 2263 samples, validate on 400 samples
Epoch 1/38
2263/2263 [==============================] - 6s 3ms/step - loss: 0.4352 - accuracy: 0.8621 - val_loss: 0.1801 - val_accuracy: 0.9850
Epoch 2/38
2263/2263 [==============================] - 4s 2ms/step - loss: 0.3812 - accuracy: 0.8736 - val_loss: 0.1749 - val_accuracy: 0.9850
Epoch 3/38
2263/2263 [==============================] - 4s 2ms/step - loss: 0.3809 - accuracy: 0.8736 - val_loss: 0.1896 - val_accuracy: 0.9850
Epoch 4/38
2263/2263 [==============================] - 4s 2ms/step - loss: 0.3802 - accuracy: 0.8736 - val_loss: 0.1635 - val_accuracy: 0.9850
Epoch 5/38
2263/2263 [==============================] - 4s 2ms/step - loss: 0.3799 - accuracy: 0.8736 - val_loss: 0.1477 - val_accuracy: 0.9850
Epoch 6/38
2263/2263 [==============================] - 4s 2ms/step - loss: 0.3797 - accuracy: 0.8736 - val_loss: 0.1631 - val_accuracy: 0.9850
Epoch 7/38
2263/2263 [==============================] - 4s 2ms/step - loss: 0.3802 - accu

In [16]:
l = [["Daily Weight" ,a_a],["Daily Length" ,a_b],["pH" ,a_c],["TPN/TFL" ,a_d],["RBS" ,a_e],["Stool" ,a_f],["Urine/kg/Hr" ,a_g]]

table = PrettyTable(['Parameter', 'AUC-ROC'])

for rec in l:
    table.add_row(rec)

print(table)

+--------------+-----------------------+
|  Parameter   |        AUC-ROC        |
+--------------+-----------------------+
| Daily Weight | (0.933, 0.929, 0.937) |
| Daily Length |  (0.79, 0.018, 1.563) |
|      pH      | (0.735, 0.732, 0.737) |
|   TPN/TFL    | (0.564, 0.467, 0.661) |
|     RBS      | (0.576, 0.469, 0.683) |
|    Stool     |  (0.64, 0.636, 0.643) |
| Urine/kg/Hr  | (0.545, 0.434, 0.656) |
+--------------+-----------------------+
